# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [1]:
%pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv
import os
from dotenv import load_dotenv
from huggingface_hub import notebook_login

load_dotenv()
HF_AGENTS_COURSE=os.getenv("HF_TOKEN")

notebook_login()

Note: you may need to restart the kernel to use updated packages.


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    
    flight_time = (actual_distance / (cruising_speed_kmh or math.nan) ) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
# from google.colab import userdata
import os
# os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

In [7]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [8]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

5. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOooKiXKL1deEOHvaqqsKCKSd7QlwXjzLpPg2JmD
dSMrUwGAHi1Uo)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

8. [The Batman (2022) Locations](https://www.latlong.net/location/the-batman-2022-locations-427)
Source: Latitude and Longitude Finder

The Batman (2022) was filmed in Liverpool and London, England, UK. Filming also took place at Cardington Studios 
and Warner Bros. Leavesden Studios.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...

Out: None

[Step 0: Duration 9.76 seconds| Input tokens: 2,399 | Output tokens: 160]

...

[Step 1: Duration 18.70 seconds| Input tokens: 5,911 | Output tokens: 1,302]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract locations and their coordinates for supercar factories                                                 
  supercar_locations = [                                                                                           
      ("Maranello, Italy (Ferrari)", (44.5403, 10.9921)),                                                          
      ("Sant'Agata Bolognese, Italy (Lamborghini)", (44.5208, 11.3264)),                                           
      ("San Cesario sul Panaro, Italy (Pagani)", (44.6517, 10.9886)),                                              
      ("Bologna, Italy (Ducati)", (44.4937, 11.3426)),                                                             
      ("Woking, United Kingdom (McLaren)", (51.3775, -0.5714)),                                                    
      ("Goodwood, United Kingdom (Aston Martin)", (50.8380, -0.7893)),                                             
      ("Enfield, United Kingdom (Lotus)", (51.6566, -0.3653)),                                                     
      ("Gordon Murray Automotive, Surrey, United Kingdom", (51.3588, -0.2191)),                                    
      ("Santa Fe Springs, California, USA (Tesla)", (33.9716, -118.0845)),                                         
      ("Los Angeles, California, USA (Koenigsegg)", (34.0522, -118.2437)),                                         
      ("Amersfoort, Netherlands (Spyker)", (52.0118, 5.6214)),                                                     
      ("Shanghai, China (Shanghai Automotive Industry Corporation)", (31.2304, 121.4737)),                         
      ("Dubai, United Arab Emirates (Emirates Motors)", (25.2769, 55.2962)),                                       
      ("Berlin, Germany (Porsche)", (52.5200, 13.4050)),                                                           
      ("Munich, Germany (BMW M)", (48.1351, 11.5820)),                                                             
      ("Zurich, Switzerland (Rimac Automobili)", (47.3667, 8.5392)),                                               
      ("Tokyo, Japan (Nissan GT-R)", (35.6828, 139.7594)),                                                         
      ("São Paulo, Brazil (Italdesign)", (-23.5505, -46.6333))                                                     
  ]                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = []                                                                                       
  for location, coords in supercar_locations:                                                                      
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times.append((location, travel_time))[48;2;39;40;

Execution logs:
                                             Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2   Knebworth House, Knebworth, Hertfordshire, Eng...   
3   Acton Lane Power Station, Acton Lane, Acton, E...   
4              Wollaton Hall, Nottingham, England, UK   
5   Carnegie Mellon University, Pittsburgh, Pennsy...   
6          The Farmiloe Building, London, England, UK   
7     Queensboro Bridge, New York City, New York, USA   
8           Wall Street, New York City, New York, USA   
9          Mehrangarh Fort, Jodhpur, Rajasthan, India   
10                        Turda Gorge, Turda, Romania   
11                             Chicago, Illinois, USA   
12             Westwood, Los Angeles, California, USA   
13  Cardington Studios, Northamptonshire, England, UK   
14  Warner Bros. Leavesden Studios, Hertfordshire,...   
15   Gordon Murray Automotive, Surrey, United Kingdom   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.15  
3                            9.16  
4                            9.00  
5                            1.74  
6                            9.17  
7                            1.01  
8                            1.00  
9                           18.34  
10                          11.87  
11                           2.68  
12                           6.79  
13                           9.10  
14                           9.15  
15                           9.17  

Out - Final answer:                                              Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2   Knebworth House, Knebworth, Hertfordshire, Eng...   
3   Acton Lane Power Station, Acton Lane, Acton, E...   
4              Wollaton Hall, Nottingham, England, UK   
5   Carnegie Mellon University, Pittsburgh, Pennsy...   
6          The Farmiloe Building, London, England, UK   
7     Queensboro Bridge, New York City, New York, USA   
8           Wall Street, New York City, New York, USA   
9          Mehrangarh Fort, Jodhpur, Rajasthan, India   
10                        Turda Gorge, Turda, Romania   
11                             Chicago, Illinois, USA   
12             Westwood, Los Angeles, California, USA   
13  Cardington Studios, Northamptonshire, England, UK   
14  Warner Bros. Leavesden Studios, Hertfordshire,...   
15   Gordon Murray Automotive, Surrey, United Kingdom   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.15  
3                            9.16  
4                            9.00  
5                            1.74  
6                            9.17  
7                            1.01  
8                            1.00  
9                           18.34  
10                          11.87  
11                           2.68  
12                           6.79  
13                           9.10  
14                           9.15  
15                           9.17  

[Step 2: Duration 18.27 seconds| Input tokens: 12,237 | Output tokens: 2,588]

In [9]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
3,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
4,"Wollaton Hall, Nottingham, England, UK",9.00
5,"Carnegie Mellon University, Pittsburgh, Pennsy...",1.74
6,"The Farmiloe Building, London, England, UK",9.17
7,"Queensboro Bridge, New York City, New York, USA",1.01
8,"Wall Street, New York City, New York, USA",1.00
9,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [10]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Calculate the cargo plane transfer time from each filming location to Gotham.
4. Perform a web search to find a comprehensive list of supercar factories worldwide.
5. For each supercar factory, visit the source URL to confirm the coordinates.
6. Calculate the cargo plane transfer time from each supercar factory to Gotham.
7. Filter both lists to include only locations with the same cargo plane transfer time.
8. Organize the filtered locations and their details into a pandas dataframe.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                             Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2   Knebworth House, Knebworth, Hertfordshire, Eng...   
3   Acton Lane Power Station, Acton Lane, Acton, E...   
4              Wollaton Hall, Nottingham, England, UK   
5   Carnegie Mellon University, Pittsburgh, Pennsy...   
6          The Farmiloe Building, London, England, UK   
7     Queensboro Bridge, New York City, New York, USA   
8           Wall Street, New York City, New York, USA   
9          Mehrangarh Fort, Jodhpur, Rajasthan, India   
10                        Turda Gorge, Turda, Romania   
11                             Chicago, Illinois, USA   
12             Westwood, Los Angeles, California, USA   
13  Cardington Studios, Northamptonshire, England, UK   
14  Warner Bros. Leavesden Studios, Hertfordshire,...   
15   Gordon Murray Automotive, Surrey, United Kingdom   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.15  
3                            9.16  
4                            9.00  
5                            1.74  
6                            9.17  
7                            1.01  
8                            1.00  
9                           18.34  
10                          11.87  
11                           2.68  
12                           6.79  
13                           9.10  
14                           9.15  
15                           9.17

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
  import pandas as pd                                                                                              
                                                                                                                   
  # Function to extract locations and their coordinates                                                            
  def extract_locations(content):                                                                                  
      locations = []                                                                                               
      # Regex to find locations and their coordinates                                                              
      location_pattern = re.compile(r'([A-Za-z\s,]+)\s*\(([-+]?\d+\.\d+),\s*([-+]?\d+\.\d+)\)')                    
      matches = location_pattern.findall(content)                                                                  
      for match in matches:                                                                                        
          locations.append((match[0].strip(), float(match[1]), float(match[2])))                                   
      return locations                                                                                             
                                                                                                                   
  # Extract locations from each source                                                                             
  all_locations = []                                                                                               
  for url, content in batman_locations:                                                                            
      locations = extract_locations(content)                                                                       
      all_locations.extend(locations)                                                                              
                                                                                                                   
  # If no coordinates found, perform additional web searches                                                       
  if not all_locations:                                                                                            
      locations_to_search = [                                                                                      
          "Necropolis Cemetery, Glasgow, Scotland, UK",                                                            
          "St. George's Hall, Liverpool, England, UK",                                                             
          "Knebworth House, Knebworth, Hertfordshire, England, UK",                                                
          "Acton Lane Power Station, Acton Lane, Acton, England, UK",                                              
          "Wollaton Hall, Nottingham, England, UK",                                                                
          "Carnegie Mellon University, Pittsburgh, Pennsylvania, USA",                                             
          "The Farmiloe Building, London, England, UK",                                                            
          "Queensboro Bridge, New York City, New York, USA",                                                       
          "Wall Street, New York City, New York, USA",                                                             
          "Mehrangarh Fort, Jodhpur, Rajasthan, India",                                                            
          "Turda Gorge, Turda, Romania",                                                                           
          "Chicago, Illinois, USA",                      

Code execution failed at line 'for url, content in batman_locations:
    locations = extract_locations(content)
    all_locations.extend(locations)' due to: InterpreterError: Object <module 're' from 
'/Users/fperez/.pyenv/versions/3.11.5/lib/python3.11/re/__init__.py'> has no attribute compile

[Step 3: Duration 21.84 seconds| Input tokens: 33,820 | Output tokens: 2,237]

[Step 4: Duration 0.12 seconds| Input tokens: 56,585 | Output tokens: 3,658]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67c0f061-7982c2316dec98dc0d6e8140;b430b2c3-6cfc-4514-8172-13d4bb274233)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.

In [11]:
detailed_report

NameError: name 'detailed_report' is not defined

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [13]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(),  # provider="serper"
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
# from google.colab import userdata
import os
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [14]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [15]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [16]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

[Step 0: Duration 0.18 seconds]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67c0f23a-1aa885e217f7f11b4917644a;4839b886-35c9-43b1-9047-3dfb36da7d6b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)